In [ ]:
!pip install -U sentence-transformers
#!pip install "gensim==3.8.1"
#!pip install texthero
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
#import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
#import flair
from scipy.spatial import distance
#from simpletransformers.language_representation import RepresentationModel
import requests
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#import umap.umap_ as umap
from sklearn.decomposition import PCA
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import silhouette_score
from wordcloud import WordCloud
#import texthero as hero
from sklearn import metrics
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')
data_train.drop(['ArticleId'],inplace=True,axis=1)
data_train.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [ ]:
sentences=data_train['Text'].to_list()

In [ ]:
model_name='bert-base-nli-mean-tokens'
from sentence_transformers import SentenceTransformer
model=SentenceTransformer(model_name)
sentence_vecs=model.encode(sentences)

KeyboardInterrupt: ignored

In [ ]:
embedder=SentenceTransformer('bert-base-nli-mean-tokens')
def get_embedding(text):
    return embedder.encode(text)
data_train['embedding']=data_train['Text'].apply(get_embedding)
features=data_train['embedding'].to_list()

In [ ]:
!pip install fuzzy-c-means

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.1 MB 12.1 MB/s 


In [ ]:
from fcmeans import FCM
features=np.array(features)
my_model = FCM(n_clusters=5,random_state=42) 
my_model.fit(features)

In [ ]:
data_train['cluster']=my_model.predict(features)
data_train.head(10)

,Text,Category,embedding,cluster
0,worldcom ex-boss launches defence lawyers defe...,business,"[-0.3872127, 1.084979, -0.3821319, 0.04942725,...",4
1,german business confidence slides german busin...,business,"[-0.9098999, 0.73046386, -0.03905958, -0.23986...",0
2,bbc poll indicates economic gloom citizens in ...,business,"[-0.7895384, 0.7858662, -0.64425606, -0.005489...",4
3,lifestyle governs mobile choice faster bett...,tech,"[-0.13440779, 0.707621, 0.6627281, 0.026215313...",4
4,enron bosses in $168m payout eighteen former e...,business,"[-0.71415865, 0.55227387, -0.9366001, -0.15856...",0
5,howard truanted to play snooker conservative...,politics,"[-0.41936463, 0.7185373, 0.3620656, 0.03688574...",4
6,wales silent on grand slam talk rhys williams ...,sport,"[-0.15132383, 0.45348868, 1.1249807, -0.810152...",4
7,french honour for director parker british film...,entertainment,"[-0.34462366, 0.8080942, -0.16235378, 0.074576...",0
8,car giant hit by mercedes slump a slump in pro...,business,"[-0.6687432, 0.774405, -0.77372587, -0.3714481...",0
9,fockers fuel festive film chart comedy meet th...,entertainment,"[-0.84819597, 0.703826, -0.25610527, -0.083271...",0


In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 0, Number of samples: 210
tech:
 Top cluster number: 0, Number of samples: 152
politics:
 Top cluster number: 4, Number of samples: 238
sport:
 Top cluster number: 4, Number of samples: 179
entertainment:
 Top cluster number: 0, Number of samples: 172

Map cluster number to category:


{0: 'entertainment', 4: 'sport'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)
data_train.head(20)

,Text,Category,embedding,cluster,clustered_category
0,worldcom ex-boss launches defence lawyers defe...,business,"[-0.3872127, 1.084979, -0.3821319, 0.04942725,...",4,sport
1,german business confidence slides german busin...,business,"[-0.9098999, 0.73046386, -0.03905958, -0.23986...",0,entertainment
2,bbc poll indicates economic gloom citizens in ...,business,"[-0.7895384, 0.7858662, -0.64425606, -0.005489...",4,sport
3,lifestyle governs mobile choice faster bett...,tech,"[-0.13440779, 0.707621, 0.6627281, 0.026215313...",4,sport
4,enron bosses in $168m payout eighteen former e...,business,"[-0.71415865, 0.55227387, -0.9366001, -0.15856...",0,entertainment
5,howard truanted to play snooker conservative...,politics,"[-0.41936463, 0.7185373, 0.3620656, 0.03688574...",4,sport
6,wales silent on grand slam talk rhys williams ...,sport,"[-0.15132383, 0.45348868, 1.1249807, -0.810152...",4,sport
7,french honour for director parker british film...,entertainment,"[-0.34462366, 0.8080942, -0.16235378, 0.074576...",0,entertainment
8,car giant hit by mercedes slump a slump in pro...,business,"[-0.6687432, 0.774405, -0.77372587, -0.3714481...",0,entertainment
9,fockers fuel festive film chart comedy meet th...,entertainment,"[-0.84819597, 0.703826, -0.25610527, -0.083271...",0,entertainment


In [ ]:
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.23557046979865773


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(features,data_train['cluster']))

Silhouette Coefficient: -0.020


#Sentence-Bert Preprocessed Data + C-Means

In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import gensim
from gensim.utils import simple_preprocess
from sklearn.cluster import KMeans
from gensim.models import Word2Vec
from sklearn import cluster
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from nltk.cluster import KMeansClusterer, euclidean_distance
import nltk
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import DBSCAN
import seaborn as sns
from wordcloud import WordCloud
from nltk.tokenize import wordpunct_tokenize
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
nltk.download('stopwords') 
nltk.download('wordnet')
nltk.download('omw-1.4')
data_train=pd.read_csv('/content/drive/MyDrive/BBC News Train.csv')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


In [ ]:
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
data_train.drop(['ArticleId'],axis=1,inplace=True)

In [ ]:
def expand_contractions(text):
    expanded_words = []    
    for word in text.split():
        expanded_words.append(contractions.fix(word))   
    expanded_text = ' '.join(expanded_words)
    return expanded_text
 
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
def remove_digits(text):
    return " ".join([word for word in str(text).split() if not(word.isdigit())])
def remove_shorttokens(text):
    return " ".join([word for word in str(text).split() if len(word)>1])
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
#convert it to string
data_train["Text"]=data_train["Text"].astype(str)
#Convert to lower and strip
data_train["Text"]=data_train["Text"].str.lower().str.strip()
#apply contractions
data_train["Text"]=data_train["Text"].apply(lambda x:expand_contractions(x))
#remove punctuations
data_train["Text"]=data_train["Text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '' , x))
#remove stop words
data_train["Text"]=data_train["Text"].apply(lambda x: remove_stopwords(x))
#Lemmatize the sentence
data_train["Text"]=data_train["Text"].apply(lambda text: lemmatize_words(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_digits(text))
data_train["Text"]=data_train["Text"].apply(lambda text:remove_shorttokens(text))

In [ ]:
sentences=data_train['Text'].to_list()
model_name='bert-base-nli-mean-tokens'
from sentence_transformers import SentenceTransformer
model=SentenceTransformer(model_name)
sentence_vecs=model.encode(sentences)

In [ ]:
from fcmeans import FCM
my_model = FCM(n_clusters=5,random_state=42) 
my_model.fit(sentence_vecs)

In [ ]:
data_train['cluster']=my_model.predict(sentence_vecs)
data_train.head(10)

,Text,Category,cluster
0,worldcom exboss launch defence lawyer defendin...,business,4
1,german business confidence slide german busine...,business,4
2,bbc poll indicates economic gloom citizen majo...,business,4
3,lifestyle governs mobile choice faster better ...,tech,3
4,enron boss 168m payout eighteen former enron d...,business,4
5,howard truanted play snooker conservative lead...,politics,4
6,wale silent grand slam talk rhys williams say ...,sport,2
7,french honour director parker british film dir...,entertainment,0
8,car giant hit mercedes slump slump profitabili...,business,3
9,fockers fuel festive film chart comedy meet fo...,entertainment,3


In [ ]:
cluster_to_category = {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category

business:
 Top cluster number: 4, Number of samples: 223
tech:
 Top cluster number: 4, Number of samples: 113
politics:
 Top cluster number: 4, Number of samples: 252
sport:
 Top cluster number: 2, Number of samples: 171
entertainment:
 Top cluster number: 0, Number of samples: 119

Map cluster number to category:


{0: 'entertainment', 2: 'sport', 4: 'politics'}

In [ ]:
data_train['clustered_category'] = data_train['cluster'].map(cluster_to_category)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category']))

Overall accuracy of clustered categories: 0.363758389261745


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(sentence_vecs,data_train['cluster']))

Silhouette Coefficient: 0.043


#Experiment with PCA

In [ ]:
data_embbedding=sentence_vecs
x=[]
y=[]
for i in range(768,0,-10):
  pca_2 = PCA(n_components=i)
  pca_2_result = pca_2.fit_transform(data_embbedding)
  x.append(i)
  y.append(round(np.sum(pca_2.explained_variance_ratio_),2))
  print('Cumulative variance explained by {} principal components: {:.2%}'.format(i,np.sum(pca_2.explained_variance_ratio_)))

Cumulative variance explained by 768 principal components: 100.00%
Cumulative variance explained by 758 principal components: 100.00%
Cumulative variance explained by 748 principal components: 100.00%
Cumulative variance explained by 738 principal components: 99.99%
Cumulative variance explained by 728 principal components: 99.99%
Cumulative variance explained by 718 principal components: 99.99%
Cumulative variance explained by 708 principal components: 99.98%
Cumulative variance explained by 698 principal components: 99.98%
Cumulative variance explained by 688 principal components: 99.97%
Cumulative variance explained by 678 principal components: 99.97%
Cumulative variance explained by 668 principal components: 99.96%
Cumulative variance explained by 658 principal components: 99.95%
Cumulative variance explained by 648 principal components: 99.95%
Cumulative variance explained by 638 principal components: 99.94%
Cumulative variance explained by 628 principal components: 99.93%
Cumulat

In [ ]:
##95%,90%,85% covaience-->145,85,62
data_embbedding=sentence_vecs
pca_95 = PCA(n_components=145)
pca_95result = pca_95.fit_transform(data_embbedding)

pca_90 = PCA(n_components=85)
pca_90result = pca_95.fit_transform(data_embbedding)

pca_85 = PCA(n_components=65)
pca_85result = pca_85.fit_transform(data_embbedding)

In [ ]:
from fcmeans import FCM
my_model95= FCM(n_clusters=5,random_state=42) 
my_model95.fit(pca_95result)
data_train['cluster95']=my_model95.predict(pca_95result)

my_model90= FCM(n_clusters=5,random_state=42) 
my_model90.fit(pca_90result)
data_train['cluster90']=my_model95.predict(pca_90result)

my_model85= FCM(n_clusters=5,random_state=42) 
my_model85.fit(pca_85result)
data_train['cluster85']=my_model85.predict(pca_85result)



In [ ]:
cluster_to_category95= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster95'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category95[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category95

business:
 Top cluster number: 4, Number of samples: 225
tech:
 Top cluster number: 4, Number of samples: 113
politics:
 Top cluster number: 4, Number of samples: 252
sport:
 Top cluster number: 2, Number of samples: 159
entertainment:
 Top cluster number: 0, Number of samples: 130

Map cluster number to category:


{0: 'entertainment', 2: 'sport', 4: 'politics'}

In [ ]:
data_train['clustered_category95'] = data_train['cluster'].map(cluster_to_category95)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category95']))

Overall accuracy of clustered categories: 0.363758389261745


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(pca_95result,data_train['cluster95']))

Silhouette Coefficient: 0.045


In [ ]:
cluster_to_category90= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster90'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category90[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category90

business:
 Top cluster number: 4, Number of samples: 225
tech:
 Top cluster number: 4, Number of samples: 113
politics:
 Top cluster number: 4, Number of samples: 252
sport:
 Top cluster number: 2, Number of samples: 159
entertainment:
 Top cluster number: 0, Number of samples: 130

Map cluster number to category:


{0: 'entertainment', 2: 'sport', 4: 'politics'}

In [ ]:
data_train['clustered_category90'] = data_train['cluster'].map(cluster_to_category90)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category90']))

Overall accuracy of clustered categories: 0.363758389261745


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(pca_90result,data_train['cluster90']))

Silhouette Coefficient: 0.045


In [ ]:
cluster_to_category85= {}
for cat in data_train['Category'].unique():
    mark = data_train['Category'] == cat
    top = data_train[mark]['cluster85'].value_counts().head(1)
    count = top.values[0]
    cluster = top.index[0]
    print(f"{cat}:\n Top cluster number: {cluster}, Number of samples: {count}")
    cluster_to_category85[cluster] = cat

print("\nMap cluster number to category:")
cluster_to_category85

business:
 Top cluster number: 4, Number of samples: 226
tech:
 Top cluster number: 4, Number of samples: 113
politics:
 Top cluster number: 4, Number of samples: 252
sport:
 Top cluster number: 2, Number of samples: 139
entertainment:
 Top cluster number: 0, Number of samples: 143

Map cluster number to category:


{0: 'entertainment', 2: 'sport', 4: 'politics'}

In [ ]:
data_train['clustered_category85'] = data_train['cluster'].map(cluster_to_category85)
print('Overall accuracy of clustered categories:', np.mean(data_train['Category'] == data_train['clustered_category85']))

Overall accuracy of clustered categories: 0.363758389261745


In [ ]:
from sklearn.metrics import silhouette_score
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(pca_85result,data_train['cluster85']))

Silhouette Coefficient: 0.047
